# Pre-processing of chSPSIM data

In [1]:
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#import statistics
#from PIL import Image
from scipy.linalg import hadamard
import math 
from matplotlib.colors import LogNorm
from sympy.combinatorics.graycode import GrayCode
from scipy.ndimage.filters import gaussian_filter
from scipy.stats import sigmaclip
from scipy import sparse
import scipy.sparse.linalg
from scipy.sparse import linalg
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display


sys.path.append('/home/crombez/Documents/PhD/python/libreries/') 
from data_visualisation import *
from load_data import *
from matrix_tools import *
from walsh_hadamard import *
from walsh_ordered_recon import *

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
def bining_line(Mat,n):
    (Nl,Nc) = np.shape(Mat)
    M_out = np.zeros((Nl//n,Nc))
    for i in range(0,Nl,n):
        for j in range(n):
            M_out[i//n] += Mat[i+j]
    return(M_out)

def bining_colonne(Mat,n):
    (Nl,Nc) = np.shape(Mat)
    M_out = np.zeros((Nl,Nc//n))
    for i in range(0,Nc,n):
        for j in range(n):
            M_out[:,i//n] += Mat[:,i+j]
    return(M_out)

In [4]:
def Files_names(Path,name_type):
    files = glob.glob(Path+name_type)
    print
    files.sort(key=os.path.getmtime)
    return([os.path.basename(x) for x in files])

In [5]:
def Select_data(Dir,Run):
    Data_path = Dir+Run+'/'
    name_type = os.listdir(Data_path)[0]
    name_type = name_type[:-9]+'*'+name_type[-4:]
    files = Files_names(Data_path,str(name_type))
    print('Données du dossier : '+Data_path)

    return(Data_path,files)

In [6]:
def load_data(Path_files,list_files,Nl,Nh,Nc):
    Data = np.zeros((Nl,Nh,Nc))
    
    for i in range(0,2*Nh,2):
        
        Data[:,i//2] = np.float_(np.rot90(np.array(Image.open(Path_files+list_files[i]))))-np.float_(np.rot90(np.array(Image.open(Path_files+list_files[i+1]))))

    return(Data)

## Hadamard acquisition

### Data chSPSIM

In [8]:
Dir = '/run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_13/'
list_run = ['RUN0002','RUN0003','RUN0004','RUN0005','RUN0006',]
Nl = 2160 # number of lines of the spectral camera
Nc = 2560 # number of columns of the spectral camera
Nh = 128#64#  #number of patterns acquired

Bin_line = 4#8#
Bin_lambda = 20

Nl_bin = 2048//Bin_line
Nc_bin = Nc//Bin_lambda
n = 1
for Run in list_run :
    Data_path,files = Select_data(Dir,Run)
    data = load_data(Data_path,files,Nl,Nh,Nc)
    stack = np.zeros((Nl_bin,Nh,Nc_bin))
    for i in range(Nh):
        step = bining_colonne(data[56:2104,i],Bin_lambda) # Here as we only have 2048 lines on the imaging camera we remove 56 lines at the top and the bottom of the spectrale images 
        stack[:,i] = bining_line(step,Bin_line)
    save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_3D/Preprocess/'
    #name = Run+'_Had_'+str(Nl_bin)+'_'+str(Nh)+'_'+str(Nc_bin)+'.npy'
    name = 'T'+str(n)+'_'+Run+'_2023_03_13'+'_Had_'+str(Nl_bin)+'_'+str(Nh)+'_'+str(Nc_bin)+'.npy'
    n += 1
    np.save(save_path+name,stack)

Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_13/RUN0002/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_13/RUN0003/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_13/RUN0004/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_13/RUN0005/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_13/RUN0006/


In [9]:
Dir = '/run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/'
list_run = ['RUN0001','RUN0002','RUN0003','RUN0004','RUN0005','RUN0006','RUN0007','RUN0008','RUN0009','RUN0010','RUN0011','RUN0012','RUN0013','RUN0014','RUN0015','RUN0016','RUN0017','RUN0018','RUN0019','RUN0020','RUN0021']

Nl = 2160 # number of lines of the spectral camera
Nc = 2560 # number of columns of the spectral camera
Nh = 128#64#  #number of patterns acquired

Bin_line = 4#8#
Bin_lambda = 20

Nl_bin = 2048//Bin_line
Nc_bin = Nc//Bin_lambda
n = 1
for Run in list_run :
    Data_path,files = Select_data(Dir,Run)
    data = load_data(Data_path,files,Nl,Nh,Nc)
    stack = np.zeros((Nl_bin,Nh,Nc_bin))
    for i in range(Nh):
        step = bining_colonne(data[56:2104,i],Bin_lambda) # Here as we only have 2048 lines on the imaging camera we remove 56 lines at the top and the bottom of the spectrale images 
        stack[:,i] = bining_line(step,Bin_line)
    save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_3D/Preprocess/'
    #name = Run+'_Had_'+str(Nl_bin)+'_'+str(Nh)+'_'+str(Nc_bin)+'.npy'
    name = 'T'+str(n+5)+'_'+Run+'_2023_03_14'+'_Had_'+str(Nl_bin)+'_'+str(Nh)+'_'+str(Nc_bin)+'.npy'
    n += 1
    np.save(save_path+name,stack)

Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0001/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0002/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0003/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0004/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0005/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0006/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0007/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0008/
Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_03_14/RUN0009/
Données du dossier : /run/media/crombez/Elements/HSPIM_

### Patterns

In [14]:
Dir = '/run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_02_28/'
Run = 'RUN0002'
Data_path,files = Select_data(Dir,Run)

Données du dossier : /run/media/crombez/Elements/HSPIM_seb_acquisition/Post_doc/data_2023_02_28/RUN0002/


In [15]:
%%time
Nl = 2048 # number of lines of the imaging camera
Nc = 2048 # number of columns of the imaging camera
Nh = 128#64# #number of patterns acquired
Bin_col = Bin_line
data = load_data(Data_path,files,Nl,Nh,Nc)

CPU times: user 8.14 s, sys: 10 s, total: 18.2 s
Wall time: 48.5 s


mean pattern around the middle of the field of view

In [16]:
motifs = np.zeros((Nh,Nc))
for i in range(Nh):
    motifs[i] = np.sum(data[1000:1048,i],0)
motifs = bining_colonne(motifs,Bin_col)
plot_im2D(motifs)

In [17]:
save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_3D/Reconstruction/Mat_rc/'
name = "motifs_Hadamard_"+str(Nh)+"_"+str(Nc//Bin_col)+".npy"
np.save(save_path+name,motifs)